In [ ]:
CREATE DATABASE A4;

In [ ]:
use A4;

In [ ]:
CREATE TABLE GroupTable(
    gid INT PRIMARY KEY,
    gname VARCHAR(255) NOT NULL
);


In [ ]:
CREATE TABLE StudentTable(
    sid INT PRIMARY KEY,
    sname VARCHAR(30),
    gid INT FOREIGN KEY REFERENCES GroupTable(gid)
);

In [ ]:
CREATE TABLE FavSubjectTable(
    sid INT FOREIGN KEY REFERENCES StudentTable(sid),
    gid INT FOREIGN KEY REFERENCES GroupTable(gid),
    subjectName VARCHAR(50),
    PRIMARY KEY(sid, gid)
)

In [ ]:
CREATE PROCEDURE InsertGroups (@NoOfRows INT) -- ex. insert 500 groups
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    DECLARE @i INT = 1;
    -- Loop to insert 200 groups
    WHILE @i <= @NoOfRows
    BEGIN
        -- Insert statement
        INSERT INTO GroupTable (gid, gname) VALUES (
            @i,
            CONCAT('Group_', @i) -- Generating a unique name "Group_1", "Group_2", etc.
        ) 
        -- Increment the student ID counter
        SET @i = @i + 1;
    END;
END;

In [ ]:
CREATE PROCEDURE InsertStudents (@NoOfRows INT) -- ex. insert 10.000 students
AS 
BEGIN
    SET NOCOUNT ON; -- mute messages
    DECLARE @i INT = 1;
    DECLARE @groupNum INT = 1; -- get the number of groups
    DECLARE @NoOfGroups INT;
    SELECT @NoOfGroups = COUNT(*) FROM GroupTable;
    -- Loop to insert 10,000 students
    WHILE @i <= @NoOfRows
    BEGIN
        -- Insert statement
        INSERT INTO StudentTable (sid, sname, gid) VALUES (
            @i,
            CONCAT('Student_', @i),  -- Generating a unique name "Student_1", "Student_2", etc.
            @groupNum
        );
        
        -- Increment the student ID counter
        SET @i = @i + 1;

        -- Cycle through group IDs from 1 to 500
        SET @groupNum = @groupNum + 1;
        IF @groupNum > @NoOfGroups 
        BEGIN
            SET @groupNum = 1;
        END;
    END;
END;

In [ ]:
CREATE PROCEDURE InsertFavSubjects (@NoOfRows INT) -- ex. insert 2.000 fav subjects for the first 2.000 students (@NoOfRows should be less than the total number of students)
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    DECLARE @i INT = 1;
    DECLARE @groupNum INT = 1;
    DECLARE @NoOfGroups INT;
    SELECT @NoOfGroups = COUNT(*) FROM GroupTable;

    WHILE @i <= @NoOfRows
    BEGIN
        DECLARE @subjectName VARCHAR(50);
        IF @i % 4 = 0 
        BEGIN
            SET @subjectName = 'Math';
        END
        ELSE IF @i % 4 = 1 
        BEGIN
            SET @subjectName = 'English';
        END
        ELSE IF @i % 4 = 2
        BEGIN
            SET @subjectName = 'History';
        END
        ELSE IF @i % 4 = 3
        BEGIN
            SET @subjectName = 'Science';
        END;
        
        -- Insert statement
        INSERT INTO FavSubjectTable (sid, gid, subjectName) VALUES (
            @i,
            @groupNum,
            @subjectName  -- cycle through Math, English, History, Science
        );
        
        -- Increment the student ID counter
        SET @i = @i + 1;

        -- Cycle through group IDs from 1 to 500
        SET @groupNum = @groupNum + 1;
        IF @groupNum > @NoOfGroups
        BEGIN
            SET @groupNum = 1;
        END;
    END;
END;

In [ ]:
CREATE PROCEDURE InsertIntoTable (@tableId INT, @NoOfRows INT)
AS
BEGIN
    IF @tableId = 1 -- insert data into the GroupTable
        EXEC InsertGroups @NoOfRows
    ELSE IF @tableId = 2 -- insert data into the StudentTable
        EXEC InsertStudents @NoOfRows
    ELSE IF @tableId = 3 -- insert data into the FavSubjectTable
        EXEC InsertFavSubjects @NoOfRows
END;

In [ ]:
CREATE PROCEDURE InsertAll -- we will not use it, just made it for better convenience
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    EXEC InsertGroups;
    EXEC InsertStudents;
    EXEC InsertFavSubjects;
END;

In [ ]:
CREATE PROCEDURE ResetTables
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    DELETE FROM FavSubjectTable;    
    DELETE FROM StudentTable;
    DELETE FROM GroupTable;
    
END;

In [ ]:
CREATE FUNCTION ufIsOkToInsertFavSubjects (@NoOfFavSubjectsToInsert INT) -- function that checks if it is ok to insert the number of favorite subjects
-- THIS FUNCTION IS NOT USED IN THE CODE, BUT IT IS A GOOD EXAMPLE OF HOW TO CREATE A FUNCTION IN SQL
RETURNS BIT AS 
BEGIN
    DECLARE @result BIT;
    DECLARE @NoOfStudents INT;
    SELECT @NoOfStudents = COUNT(*) FROM StudentTable;
    IF @NoOfFavSubjectsToInsert > @NoOfStudents
        SET @result = 0;
    ELSE
        SET @result = 1;
    RETURN @result;
END;

In [ ]:
DECLARE @IsOkToInsert BIT; -- test code
SET @IsOkToInsert = dbo.ufIsOkToInsertFavSubjects(10);
PRINT @IsOkToInsert;

In [ ]:
CREATE VIEW ScienceAsFavSubject AS -- view with a SELECT statement operating on one table
    SELECT sid, gid FROM FavSubjectTable WHERE subjectName = 'Science';

In [ ]:
CREATE VIEW StudentsFromGroup1 AS -- view with a SELECT statement that operates on at least 2 different tables and contains at least one JOIN operator
    SELECT 
        StudentTable.sid,
        StudentTable.sname,
        GroupTable.gname
    FROM 
        StudentTable
    INNER JOIN 
        GroupTable ON StudentTable.gid = GroupTable.gid AND GroupTable.gid = 1;


In [ ]:
CREATE VIEW FavSubjectCount AS -- a view with SELECT statement that has a GROUP BY clause, operates on at least 2 different tables and uses at least one JOIN operator
SELECT 
    FavSubjectTable.subjectName,
    COUNT(*) AS StudentCount
FROM 
    FavSubjectTable
INNER JOIN 
    StudentTable ON StudentTable.sid = FavSubjectTable.sid
GROUP BY 
    FavSubjectTable.subjectName;



In [ ]:
CREATE TABLE Tables(
    tid INT PRIMARY KEY,
    tname VARCHAR(255) NOT NULL
);

In [ ]:
INSERT INTO Tables (tid, tname) VALUES 
    (1, 'GroupTable'),
    (2, 'StudentTable'),
    (3, 'FavSubjectTable');

In [ ]:
CREATE TABLE Tests (
    testId INT PRIMARY KEY,
    testName VARCHAR(255),
)

In [ ]:
INSERT INTO Tests VALUES 
    (1, 'Test 1');

In [ ]:
CREATE TABLE TestTables (
    testId INT FOREIGN KEY REFERENCES Tests(testId),
    tableId INT FOREIGN KEY REFERENCES Tables(tid),
    NoOfRows INT NOT NULL,
    Position INT NOT NULL,
    PRIMARY KEY (testId, tableId)
);

In [ ]:
INSERT INTO TestTables VALUES 
    (1, 1, 500, 3),
    (1, 2, 10000, 2),
    (1, 3, 2000, 1);

In [ ]:
CREATE TABLE Views (
    viewId INT PRIMARY KEY,
    viewName VARCHAR(255) NOT NULL
);

In [ ]:
INSERT INTO Views VALUES
    (1, 'ScienceAsFavSubject'),
    (2, 'StudentsFromGroup1'),
    (3, 'FavSubjectCount');

In [ ]:
CREATE TABLE TestViews (
    testId INT FOREIGN KEY REFERENCES Tests(testId),
    viewId INT FOREIGN KEY REFERENCES Views(viewId),
);

In [ ]:
INSERT INTO TestViews VALUES 
    (1, 1),
    (1, 2),
    (1, 3);

In [ ]:
CREATE TABLE TestRuns (
    testRunId INT PRIMARY KEY IDENTITY(1, 1), -- Primary key which auto increments so we don't need to keep track of it
    testId INT FOREIGN KEY REFERENCES Tests(testId),
    StartAt DATETIME,
    EndAt DATETIME,
    duration AS DATEDIFF(SECOND, StartAt, EndAt)
)

In [ ]:
CREATE TABLE TestRunTables (
    testRunId INT PRIMARY KEY IDENTITY(1, 1),
    testId INT FOREIGN KEY REFERENCES Tests(testId),
    tableId INT FOREIGN KEY REFERENCES Tables(tid),
    StartAt DATETIME,
    EndAt DATETIME,
    duration AS DATEDIFF(SECOND, StartAt, EndAt)
)

In [ ]:
CREATE TABLE TestRunViews (
    testRunId INT PRIMARY KEY IDENTITY(1, 1),
    testId INT FOREIGN KEY REFERENCES Tests(testId),
    viewId INT FOREIGN KEY REFERENCES Views(viewId),
    StartAt DATETIME,
    EndAt DATETIME,
    duration AS DATEDIFF(SECOND, StartAt, EndAt)
)

In [ ]:
CREATE PROCEDURE RunTest (@testId INT)
AS
BEGIN
    SET NOCOUNT ON;
    DECLARE @testStartTime DATETIME;
    DECLARE @testEndTime DATETIME;
    SET @testStartTime = GETDATE(); -- start the timer for the whole test

    -- DELETE the data from test T's tables, in the order specified by the Position field in TestTables
    -- use a cursor to take each table name and perform a delete on it
    DECLARE @tableId INT;
    DECLARE @tableName VARCHAR(255);
    DECLARE ClearTablesCursor CURSOR FOR
        SELECT tableId 
        FROM TestTables 
        WHERE testId = @testId 
        ORDER BY Position ASC;
    
    OPEN ClearTablesCursor;
    FETCH ClearTablesCursor INTO @tableId;
    WHILE @@FETCH_STATUS = 0
    BEGIN
        -- based on the tabeId, get the table name from the Tables table
        SELECT @tableName = tname FROM Tables WHERE tid = @tableId;
        EXEC('DELETE FROM ' + @tableName); -- delete all rows from the table
        FETCH ClearTablesCursor INTO @tableId;
    END;
    CLOSE ClearTablesCursor; -- close the cursor
    DEALLOCATE ClearTablesCursor; -- deallocate the cursor and all of its resources

    -- INSERT data into the test T's tables, in the reverse deletion order
    -- the number of records to insert into each table is stored in the NoOfRows field in the TestTables table
    -- use a cursor to go in reverse order and insert the data
    DECLARE @NoOfRows INT;
    -- @tableId is already declared above
    -- @tableName is already declared above
    DECLARE @insertStartAt DATETIME;
    DECLARE @insertEndAt DATETIME;
    DECLARE InsertTablesCursor CURSOR FOR
        SELECT tableId, NoOfRows
        FROM TestTables 
        WHERE testId = @testId 
        ORDER BY Position DESC;
    
    OPEN InsertTablesCursor;
    FETCH InsertTablesCursor INTO @tableId, @NoOfRows;
    WHILE @@FETCH_STATUS = 0
    BEGIN
        SET @insertStartAt = GETDATE(); -- start the timer for the insert
        EXEC InsertIntoTable @tableId, @NoOfRows; -- insert the data into the table
        SET @insertEndAt = GETDATE(); -- end the timer for the insert
        INSERT INTO TestRunTables (testId, tableId, StartAt, EndAt) VALUES (@testId, @tableId, @insertStartAt, @insertEndAt); -- record the insert, duration will be computed by itself
        FETCH InsertTablesCursor INTO @tableId, @NoOfRows;
    END
    CLOSE InsertTablesCursor; -- close the cursor
    DEALLOCATE InsertTablesCursor; -- deallocate the cursor and all of its resources

    -- EVALUATE the test T's VIEWS, no order specified
    -- use a cursor to take each view name and perform a select on it
    DECLARE @viewId INT;
    DECLARE @viewName VARCHAR(255);
    DECLARE @viewStartTime DATETIME;
    DECLARE @viewEndTime DATETIME;
    DECLARE EvaluateViewsCursor CURSOR FOR
        SELECT viewId 
        FROM TestViews 
        WHERE testId = @testId;
    
    OPEN EvaluateViewsCursor;
    FETCH EvaluateViewsCursor INTO @viewId;
    WHILE @@FETCH_STATUS = 0
    BEGIN
        -- based on the viewId, get the view name from the Views table
        SELECT @viewName = viewName FROM Views WHERE viewId = @viewId;
        SET @viewStartTime = GETDATE(); -- start the timer for the view
        EXEC('SELECT * FROM ' + @viewName); -- select all rows from the view
        SET @viewEndTime = GETDATE(); -- end the timer for the view
        INSERT INTO TestRunViews (testId, viewId, StartAt, EndAt) VALUES (@testId, @viewId, @viewStartTime, @viewEndTime); -- record the view, duration will be computed by itself
        FETCH EvaluateViewsCursor INTO @viewId;
    END;
    CLOSE EvaluateViewsCursor; -- close the cursor
    DEALLOCATE EvaluateViewsCursor; -- deallocate the cursor and all of its resources

    SET @testEndTime = GETDATE(); -- end the timer for the whole test
    INSERT INTO TestRuns (testId, StartAt, EndAt) VALUES (@testId, @testStartTime, @testEndTime); -- record the test run, duration will be computed by itself
    PRINT 'Test completed successfully';
END;

In [ ]:
EXEC RunTest 1;

In [ ]:
select * from TestRuns;

In [ ]:
select * from TestRunViews; -- get the individual view timings

In [ ]:
select * from TestRunTables; -- get the individual insert timings